In [ ]:
%pip install pytesseract
%pip install pillow
%pip install img2table

In [1]:
# 1. Использование pytesseract

import pytesseract
from PIL import Image

# загрузка изображения с текстом
doc1 = Image.open('tablica.jpg')

# распознавание текста
string = pytesseract.image_to_string(doc1, lang='rus')

# вывод распознанного текста
print(string)

Таблица сложения до 10
ЕЕ
резко зат анк зо вю |

пе [ные Г
па ыныыю ||
Е ОО О О
С О О О
О О ОО О ОО ООО




In [2]:
# 2. Использование img2table с TesseractOCR

from img2table.document import Image as Image2
from img2table.ocr import TesseractOCR
from IPython.display import display_html
from IPython.display import display

# создание OCR из TesseractOCR
ocr = TesseractOCR(n_threads=1, lang="rus")

# загрузка изображения с текстом
doc2 = Image2('tablica.jpg', detect_rotation=False)

# извлечение таблицы из изображения
extracted_table = doc2.extract_tables(ocr=ocr,
                                      implicit_rows=True,
                                      borderless_tables=False,
                                      min_confidence=0)

# извлечение таблицы из изображения в xlsx файл
doc2.to_xlsx('tables.xlsx', ocr=ocr,
                           implicit_rows=True,
                           borderless_tables=False,
                           min_confidence=10)

# перевод результата в датасет
data = extracted_table[0].df
print('Распознанная таблица в виде датасета:\n')
print(data.info())
display(data)

# отображение результата через display_html
table = extracted_table.pop()
display_html(table.html_repr(title='Распознанная таблица'), raw=True)

Распознанная таблица в виде датасета:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1 non-null      object
 1   1       1 non-null      object
 2   2       1 non-null      object
 3   3       1 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes
None


,0,1,2,3
0,None,А,О,О
1,в,None,None,None


,0,1,2,3
0,,А,О,О
1,в,,,


In [1]:
# 3. Поиск прямоугольников на изображениях

import sys
import numpy as np
import cv2 as cv

hsv_min = np.array((0, 54, 5), np.uint8)
hsv_max = np.array((187, 255, 253), np.uint8)

if __name__ == '__main__':
    fn = 'tablica.jpg' # имя файла, который будем анализировать
    img = cv.imread(fn)

    hsv = cv.cvtColor( img, cv.COLOR_BGR2HSV ) # меняем цветовую модель с BGR на HSV
    thresh = cv.inRange( hsv, hsv_min, hsv_max ) # применяем цветовой фильтр
    contours0, hierarchy = cv.findContours( thresh.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    # перебираем все найденные контуры в цикле
    for cnt in contours0:
        rect = cv.minAreaRect(cnt) # пытаемся вписать прямоугольник
        box = cv.boxPoints(rect) # поиск четырех вершин прямоугольника
        box = np.int0(box) # округление координат
        cv.drawContours(img,[box],0,(255,0,0),2) # рисуем прямоугольник

    cv.imshow('contours', img) # вывод обработанного кадра в окно

    cv.waitKey()
    cv.destroyAllWindows()